# Project

## Download Data

In [2]:
import urllib
import json
import pandas as pd
from bd_addons.interpro_data import *
from bd_addons.HmmPy import *

cur = os.getcwd()
filename_tblouts = cur + '\\data_team_1\\HMMs\HMM_C\\hmmsearch_out_C_1.tblout'
filename_domtblouts = cur + '\\data_team_1\\HMMs\HMM_C\\hmmsearch_out_C_1.domtblout'

url = "https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/pf03060?format=json"

In [3]:
metadata, entries, gt = get_data(url, 1)


# 8 - Evaluate Ability of Matching sequences
Once we made the PSI-BLAST e HMM-searches we will have some files where all the sequences are... we should:

- Find a format that we can parse (there should be options in the documentations of thevarious methods);
- Decide a threshold for the E-value and take all the hits that are below that E value;
- All the remaining sequences (call them OUT_SEQs)are the ones that, according to our model, belong to the Domain family.
- Make a Confusion Matrix comparing OUT_SEQs with the ground truth.
- compute accuracy, sensitivity, specificty etc

## HMM Searches Performance

In [28]:
parsed_hmm_t_C_df, parsed_hmm_t_M_df, parsed_hmm_t_O_df = parse_tblouts()

In [29]:
gt_acc = gt.accession.to_list()
swiss_prot = 563972
parsed_hmm_t_C_df['in_gt'] = parsed_hmm_t_C_df['target name'].apply(lambda x: split('-', x.replace('|', '-'))[1]).apply(lambda x: 1 if x in gt_acc else 0)
# print(parsed_hmm_t_C_df['in_gt'])
# parsed_hmm_t_C_df['in_gt'] = parsed_hmm_t_C_df['in_gt'].apply(lambda x: 1 if x in gt_acc else 0)
accuracy = (parsed_hmm_t_C_df['in_gt'].sum() + swiss_prot - len(parsed_hmm_t_C_df['in_gt'])) / (swiss_prot)
precision = parsed_hmm_t_C_df['in_gt'].sum() / len(parsed_hmm_t_C_df['in_gt'])
# recall = parsed_hmm_t_C_df['in_gt'].sum() / (parsed_hmm_t_C_df['in_gt'].sum() + positive in swiss_prot - hmm search)

print('accuracy', accuracy, 'precision', precision)

accuracy 0.9962303093061358 precision 0.010702652396463471


# 9 - Evualuate the ability of matching domain position

Questo si può fare solo con HMM... cfr .domtblout files...

In [4]:
parsed_hmm_domt_C_df, parsed_hmm_domt_M_df, parsed_hmm_domt_O_df = parse_domtblouts()